In [ ]:
# Cell 1: Import Libraries

import os
import zipfile
import random
import warnings
import time

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

In [ ]:
# Cell 2: Fungsi untuk mendapatkan informasi gambar
def get_image_size(path):
    unique_size = set()
    for root, _, files in os.walk(path):
        for file in files:
            if file.lower().endswith(('jpg', 'jpeg', 'png')):
                file_path = os.path.join(root, file)
                with Image.open(file_path) as img:
                    unique_size.add(img.size)
    return unique_size

def count_images_in_categories(path):
    category_counts = {}
    for category in os.listdir(path):
        category_path = os.path.join(path, category)
        if os.path.isdir(category_path):
            category_counts[category] = len(os.listdir(category_path))
    return category_counts

def display_image_info(train_dir, val_dir, test_dir):
    print("Ukuran Gambar:")
    for dir_path, label in zip([train_dir, val_dir, test_dir], ['Train', 'Val', 'Test']):
        print(f"{label} Unique Image Sizes: {get_image_size(dir_path)}")

    for dir_path, label in zip([train_dir, val_dir, test_dir], ['Train', 'Val', 'Test']):
        print(f"\nJumlah gambar di masing-masing kategori ({label}):")
        category_counts = count_images_in_categories(dir_path)
        for category, count in category_counts.items():
            print(f"{category}: {count} gambar")

In [ ]:
# Cell 3: Fungsi untuk menampilkan gambar contoh
def display_sample_image(path, title):
    print('\n' + '='*50 + ' Display Sample Image ' + '='*50 + '\n')
    plt.figure(figsize=(20, 3))
    categories = os.listdir(path)
    for i, category in enumerate(categories):
        category_path = os.path.join(path, category)
        if os.path.isdir(category_path):
            image_files = os.listdir(category_path)
            random_image = random.choice(image_files)
            image_path = os.path.join(category_path, random_image)

            img = Image.open(image_path)

            plt.subplot(1, 8, i + 1)
            plt.imshow(img)
            plt.axis('off')
            plt.title(f'{category}')

    plt.suptitle(f'{title}')
    plt.show()

In [ ]:
# Cell 4: Fungsi untuk menginisialisasi data generator
def initialize_data_generators(train_dir, val_dir, test_dir, img_height, img_width, batch_size, seed_value):
    print('\n' + '='*50 + ' Initialize Data Generator ' + '='*50 + '\n')
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    val_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=seed_value
    )
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=seed_value
    )
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=seed_value
    )

    return train_generator, val_generator, test_generator

In [ ]:
# Cell 5: Fungsi untuk membangun model
def build_model(img_height, img_width, num_classes, trainable=True):
    print('\n' + '='*50 + ' Build Model ' + '='*50 + '\n')
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    nama_arsitektur = base_model.name
    for layer in base_model.layers:
      layer.trainable = layer.trainable = trainable
      if layer.trainable:
          kondisi_layer = 'Unfreeze'
      else:
          kondisi_layer = 'Freeze'
    
    for layer in base_model.layers: 
        print(f'{layer.name}: {layer.trainable}')

    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.summary()
    return model, nama_arsitektur, kondisi_layer

In [ ]:
# Cell 6: Fungsi untuk melatih model
def train_model(model, train_generator, val_generator, learning_rate, epochs, nama_arsitektur, kondisi_layer, batch_size):
    print('\n' + '='*50 + ' Train Model ' + '='*50 + '\n')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    start_time = time.time()
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=[early_stopping],
        verbose=1
    )
    end_time = time.time()

    nama_model = f'{nama_arsitektur}_{kondisi_layer}_{learning_rate}_{batch_size}_model'
    model.save(f"{nama_model}.h5")
    model_file_size = os.path.getsize(f"{nama_model}.h5") / (1024 * 1024)
    print(f"Model saved as {nama_model}.h5")

    return history, end_time - start_time, model_file_size

In [ ]:
# Cell 7: Fungsi untuk evaluasi model
def evaluate_model(model, test_generator):
    print('\n' + '='*50 + ' Evaluating Model ' + '='*50 + '\n')
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

    predictions = model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())

    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')

    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    conf_matrix = confusion_matrix(true_classes, predicted_classes)

    return test_loss, test_accuracy, precision, recall, f1, report, conf_matrix, class_labels

In [ ]:
# Cell 8: Fungsi untuk membuat plot
def plot_results(history, stopped_epoch, conf_matrix, class_labels, nama_arsitektur, batch_size, learning_rate, kondisi_layer):
    print('\n' + '='*50 + ' Show Plot and Confusion Matrix ' + '='*50 + '\n')
    epochs_range = range(stopped_epoch)

    # Accuracy Plot
    plt.figure(figsize=(16, 10))
    plt.plot(epochs_range, history.history['accuracy'], label='Train Accuracy')
    plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylim(0, 1)
    plt.title(f"Accuracy Plot (Arsitektur: {nama_arsitektur}, Batch Size: {batch_size}, Learning Rate: {learning_rate}, Pengaturan Layer: {kondisi_layer})")
    plt.savefig(f"{nama_arsitektur}_{kondisi_layer}_{learning_rate}_{batch_size}_accuracy.png")
    plt.show()

    # Loss Plot
    plt.figure(figsize=(16, 10))
    plt.plot(epochs_range, history.history['loss'], label='Train Loss')
    plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylim(0, None)
    plt.title(f"Loss Plot (Arsitektur: {nama_arsitektur}, Batch Size: {batch_size}, Learning Rate: {learning_rate}, Pengaturan Layer: {kondisi_layer})")
    plt.savefig(f"{nama_arsitektur}_{kondisi_layer}_{learning_rate}_{batch_size}_loss.png")
    plt.show()

    # Confusion Matrix Plot
    plt.figure(figsize=(16, 10))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f"Confusion Matrix (Arsitektur: {nama_arsitektur}, Batch Size: {batch_size}, Learning Rate: {learning_rate}, Pengaturan Layer: {kondisi_layer})")
    plt.savefig(f"{nama_arsitektur}_{kondisi_layer}_{learning_rate}_{batch_size}_confusion_matrix.png")
    plt.show()


In [ ]:
# Cell 9: Memanggil semua fungsi
def run_experiment(batch_size, learning_rate, trainable):
    # Path dataset
    train_dir = 'dataset/train'
    val_dir = 'dataset/val'
    test_dir = 'dataset/test'

    # Hyperparameters
    img_height = 224
    img_width = 224
    epochs = 50

    display_image_info(train_dir, val_dir, test_dir)
    display_sample_image(train_dir, 'Sample Images from Train Directory')
    display_sample_image(val_dir, 'Sample Images from Val Directory')
    display_sample_image(test_dir, 'Sample Images from Test Directory')

    train_generator, val_generator, test_generator = initialize_data_generators(
        train_dir, val_dir, test_dir, img_height, img_width, batch_size, seed_value=42
    )

    model, nama_arsitektur, kondisi_layer = build_model(img_height, img_width, num_classes=8, trainable=trainable)
    history, training_time, model_file_size = train_model(model, train_generator, val_generator, learning_rate, epochs, nama_arsitektur, kondisi_layer, batch_size)
    stopped_epoch = len(history.history['accuracy'])

    test_loss, test_accuracy, precision, recall, f1, report, conf_matrix, class_labels = evaluate_model(model, test_generator)
    print(report)

    plot_results(history, stopped_epoch, conf_matrix, class_labels, nama_arsitektur, batch_size, learning_rate, kondisi_layer)

    metadata = {
        "Nama Arsitektur": [nama_arsitektur],
        "Kondisi Layer": [kondisi_layer],
        "Learning Rate": [learning_rate],
        "Batch Size": [batch_size],
        "Train Accuracy": [max(history.history['accuracy'])],
        "Val Accuracy": [max(history.history['val_accuracy'])],
        "Test Accuracy": [test_accuracy],
        "Train Loss": [min(history.history['loss'])],
        "Val Loss": [min(history.history['val_loss'])],
        "Test Loss": [test_loss],
        "Precision": [precision],
        "Recall": [recall],
        "F1-Score": [f1],
        "Epoch Berhenti": [stopped_epoch],
        "Waktu Pelatihan (s)": [training_time],
        "Besaran File (MB)": [model_file_size]
    }
    
    metadata_df = pd.DataFrame(metadata)
    
    metadata_df.to_csv(f"{nama_arsitektur}_{kondisi_layer}_{learning_rate}_{batch_size}_training_metadata.csv", index=False)
    
    print(f"Metadata DataFrame created and saved as {nama_arsitektur}_{kondisi_layer}_{learning_rate}_{batch_size}_training_metadata.csv")

In [ ]:
run_experiment(batch_size=128, learning_rate=1e-3, trainable=False)